<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik-neu/systemtechnik/ice-institut-fuer-computational-engineering"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN12/12-Bilder_mit_VAEs_generieren_pl.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
!pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# Bilderzeugung mit Variational Autoencodern (VAE)

This notebook contains the second code sample found in Chapter 8, Section 4 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff), 
[LiveBook](https://livebook.manning.com/book/deep-learning-with-python-second-edition/chapter-12/171)


- Das Sampling eines latenten Raums von Bildern, um völlig neue Bilder zu erzeugen oder vorhandene zu bearbeiten, gehört derzeit zu den verbreitetsten und erfolgreichsten Anwendungen kreativer KI. 
- In diesem und im folgenden Abschnitt werden wir einige allgemeine die Bilderzeugung betreffende Konzepte untersuchen sowie die Details der Implementierungen der beiden wichtigsten Verfahren auf diesem Gebiet betrachten: **Variational Autoencoders (VAEs)** und **Generative Adversarial Networks (GANs).** 

Die hier vorgestellten Verfahren sind keineswegs nur auf Bilder anwendbar – mithilfe von GANs oder VAEs ließen sich auch latente Räume von Klängen, Musikstücken oder sogar Texten entwickeln. Da in der Praxis allerdings Bilder die interessantesten Ergebnisse liefern, werden wir uns darauf konzentrieren.

- [Kunst mit KI](https://towardsdatascience.com/next-level-art-and-the-future-of-work-and-leisure-f66049112e44)
- [AI Art Gallery](http://www.aiartonline.com/)
- [obvious-art](https://obvious-art.com/)
- [Deep Fake](https://en.wikipedia.org/wiki/Deepfake)



## Sampling eines latenten Bilderraums

- Der Bilderzeugung liegt die Idee zugrunde, einen niedrigdimensionalen latenten Raum zu verwenden (naturgemäss einen Vektorraum), in dem jedem Punkt ein fotorealistisches Bild zugeordnet werden kann. 
- Dieser **latente Raum** stellt eine **Mannigfaltigkeit** für die fotorealistischen Bilder dar, d.h. eine kompakte Repräsentation der Bilder, wobei ähnliche Bilder nahgelegene Punkte innerhalb dieses latenten Raumes einnehmen.


Dem Modul, das diese Zuordnung (Einbettung) vornimmt, wird ein Punkt in diesem Vektorraum als Eingabe übergeben, und es gibt ein Bild (ein Pixelraster) aus. 
- Dieses Modul wird als **Generator** (wenn es sich um GANs handelt) bzw. als **Decodierer** (bei VAEs) bezeichnet. 
- Nach der Einrichtung eines solchen latenten Raums können entweder gezielt oder zufällig Punkte **gesampelt** werden, denen neu erzeugte Bilder zugeordnet sind.


## GAN und VAE

- **GANs** und **VAEs** sind zwei verschiedene Strategien zum Erlernen solcher latenten Bilderräume, die unterschiedliche Eigenschaften besitzen. 
- VAEs sind besonders gut für das Erlernen klar strukturierter Räume geeignet, in denen bestimmte Richtungen Achsen darstellen, die für die Variationen der Daten von Bedeutung sind (siehe Abbildung 8.10). 
- GANs können ebenfalls sehr realistisch wirkende Bilder erzeugen, aber der latente Raum, dem sie entstammen, besitzt weniger Struktur und Kontinuität.

- [MIT Introduction to Deep Learning](http://introtodeeplearning.com/2020/slides/6S191_MIT_DeepLearning_L4.pdf)
- http://introtodeeplearning.com/2020/index.html
- [Diederik P. Kingma, M. Welling: An Introduction to Variational Autoencoders](https://arxiv.org/pdf/1906.02691.pdf)


In [ ]:
from IPython.display import YouTubeVideo

id = "rZufA635dq4"
YouTubeVideo(id=id, width=640)


In [ ]:
id = "XOxxPcy5Gr4"
YouTubeVideo(id=id, width=640)


## Konzeptvektoren für das Bearbeiten von Bildern

Im Zusammenhang mit den erörterten Worteinbettungen (word embeddigs) gab es bereits einen Hinweis auf Konzeptvektoren.
Die zugrunde liegende Idee ist die gleiche:

- Bestimmte Richtungen im latenten Repräsentationsraum oder im Einbettungsraum stellen womöglich Achsen interessanter Variationen der ursprünglichen Daten dar. 
- In einem latenten »Gesichter«-Raum könnte es beispielsweise einen »Lächeln«-Vektor geben, für den Folgendes gilt: Wenn der Punkt $z$ die eingebettete Repräsentation eines bestimmten Gesichts darstellt, dann verweist der Punkt $z + l$ auf die Repräsentation desselben Gesichts, das lächelt. 
Wenn ein solcher Vektor gefunden wird, lassen sich Bilder bearbeiten, indem man sie 
1. in den latenten Raum **projiziert**, 
2. ihre Repräsentation auf sinnvolle Weise **verschiebt** und
3. die Repräsentation wieder **decodiert**. 

Im Bilderraum gibt es solche *Konzeptvektoren* für praktisch alle unabhängigen Dimensionen von Variationen. Für Gesichter sind beispielsweise Konzeptvektoren 
- zum Hinzufügen einer Sonnenbrille,
- zum Entfernen einer Brille oder
- zum Umwandeln eines männlichen Gesichts in ein weibliches denkbar.

Die folgende Abbildung zeigt ein Beispiel für einen »Lächeln«-Vektor, den Tom White von der Victoria
University School of Design in Neuseeland entdeckt hat. Zu diesem Zweck wurden VAEs mit einer aus den Gesichtern von Prominenten bestehenden Datenmenge (der sogenannten CelebA-Datenmenge) trainiert.

<img src="Bilder/laughing.jpg" width="640" align="center"/>

## Variational Autoencoders


Variational Autoencoders wurden praktisch zeitgleich von [Kingma und Welling](https://arxiv.org/abs/1312.6114) im
Dezember 2013 und von [Rezende, Mohamed und Wierstra](https://arxiv.org/abs/1401.4082) im Januar 2014 entdeckt.
- Dabei handelt es sich um ein **generatives Modell**, das besonders gut für die Aufgabe geeignet ist, Bilder mit Konzeptvektoren zu bearbeiten. 
- Das Modell ist sozusagen eine moderne Version des Autoencoders (ein Typ neuronaler Netze, die eine Eingabe in einen niedrigdimensionalen latenten Raum projizieren und wieder decodieren), die auf Konzepte des **Deep Learnings und Bayes‘sche Inferenz** zurückgreift.

- Ein klassischer Autoencoder projiziert ein Bild mit einem **Codierer** in einen **latenten Vektorraum** und wandelt es per **Decodierer** wieder in eine Ausgabe mit denselben Dimensionen wie die des ursprünglichen Bilds um (siehe Abbildung).
- Beim anschliessenden Training werden die Eingabebilder auch als Zielwerte verwendet, der Autoencoder erlernt also, die ursprünglichen Eingaben zu rekonstruieren.
- Durch verschiedene Einschränkungen der Codierung (der Ausgabe des Codierers) kann der Autoencoder dazu gebracht werden, mehr oder weniger interessante latente Repräsentationen der Daten zu erlernen. 
- Am gebräuchlichsten ist eine **dünnbesetzte (hauptsächlich Nullen) auf wenige Dimensionen beschränkte Codierung.** In diesem Fall bietet der Codierer die Möglichkeit, die **Daten zu komprimieren**.

In [ ]:
from IPython.display import Image

Image(
    "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/ANN12/Bilder/autoencoder.jpg"
)


In der Praxis liefern **klassische Autoencoder** allerdings weder besonders nützliche noch vernünftig strukturierte latente Räume. Zur Komprimierung sind sie auch kaum zu gebrauchen. Aus diesen Gründen sind sie weitgehend aus der Mode gekommen.




In [ ]:
Image(
    "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/ANN12/Bilder/classical_autoencoder.png"
)


- **VAEs** hingegen bereichern Autoencoder um ein wenig statistische Magie, die sie zum **Erlernen kontinuierlicher und hochgradig strukturierter latenter Räume zwingt**. 
- Sie haben sich als leistungsfähiges Werkzeug zur **Bilderzeugung** erwiesen.
- Anstatt ein Eingabebild als komprimierten festgelegten Code im latenten Raum abzulegen, wandelt ein VAE das Bild in die **Parameter einer statistischen Verteilung** um, nämlich in Mittelwert und Varianz. 
- Wir gehen also von der Annahme aus, dass das Eingabebild durch einen statistischen Vorgang erzeugt wurde, dessen Zufälligkeit der Codierer und der Decodierer berücksichtigen müssen. 
- Der VAE verwendet die **Parameter Mittelwert und Varianz**, um der Verteilung ein zufälliges Element zu entnehmen, und decodiert es, um die ursprüngliche Eingabe zu rekonstruieren (siehe Abbildung).



In [ ]:
Image(
    "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/ANN12/Bilder/variational_autoencoder.jpg"
)


## Reparametrisierungs-Trick

**Die Zufälligkeit dieses Vorgangs erhöht die Stabilität und erzwingt überall im latenten
Raum bedeutungsvolle Repräsentationen:**

Alle im latenten Raum gesampelten Punkte können decodiert und in gültige Ausgaben umgewandelt werden.

1. Der *Codierer* wandelt die Eingabebilder `input_img` in die beiden Parameter des latenten Repräsentationsraums `z_mean` und `z_log_variance` um.
2. Wir entnehmen der latenten Normalverteilung, von der wir annehmen, dass sie das Bild erzeugt hat, einen zufälligen Punkt $z$ durch die Zuweisung `z = z_mean + exp(z_log_variance) * epsilon`. `epsilon` ist ein Tensor, der zufällige kleine Werte enthält (reparametrization trick).
3. Der *Decodierer* ordnet diesem Punkt im latenten Raum wieder das ursprüngliche Eingabebild zu.

<img src="Bilder/vae.png" width="640"  align="center"/>


## Stetigkeit und niedrige Dimensionalität des latenten Raumes

- Da `epsilon` zufällige Werte enthält, ist durch diese Vorgehensweise gewährleistet, dass jeder Punkt in der Umgebung der Stelle des latenten Raums, an der `input_image (z_mean)` codiert ist, in ein Bild decodiert werden kann, das `input_img` ähnelt, und so erzwingt, dass der latente Raum kontinuierlich bedeutungsvoll bleibt.
- Zwei im latenten Raum nahe beieinanderliegende Punkte werden nach der Decodierung sehr ähnliche Bilder ergeben. 
- Diese **Stetigkeit und die niedrige Dimensionalität** des latenten Raums erzwingen, dass alle Richtungen im latenten Raum Achsen darstellen, die für die Variationen der Daten von Bedeutung sind. 
- Das verleiht dem latenten Raum Struktur und macht ihn besonders gut für die Bearbeitung mit Konzeptvektoren geeignet.

## Verlustfunktion

Die Parameter eines VAEs werden mit zwei Verlustfunktionen trainiert:
1. einem **Rekonstruktionsverlust**, der erzwingt, dass die decodierten Samples den ursprünglichen Eingaben entsprechen, sowie
2. einem **Regularisierungsverlust**, der es erleichtert, wohlgeformte latente Räume zu erlernen, und die Überanpassung an die Trainingsdaten verringert. 

Betrachten wir also die Keras-Implementierung eines VAEs. Schematisch sieht sie folgendermaßen aus:

1. `z_mean, z_log_variance = encoder(input_img)`
2. `z = z_mean + exp(z_log_variance) * epsilon`
3. `reconstructed_img = decoder(z)`
4. `model = Model(input_img, reconstructed_img)`

# Imports

In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.datasets import FashionMNIST, MNIST
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.utils import make_grid
import pytorch_lightning as pl
from matplotlib import pyplot as plt
import numpy as np

# %matplotlib widget


# **LitAutoEncoder – Klassifizierungs-konditionierter Variational Autoencoder (VAE)**

Dieses Modell ist ein **konditionierter VAE**, der mit PyTorch Lightning implementiert ist. Es kombiniert **Bilddaten (z. B. MNIST)** mit **Klassenzugehörigkeit (Labels)** zur Erzeugung latenter Repräsentationen und Rekonstruktionen.


##### `one_hot_embedding(labels, num_classes)`
Hilfsfunktion zur Umwandlung von Labels in One-Hot-Vektoren mit `torch.eye`.


### Architektur – `LitAutoEncoder`

#### Encoder
- Besteht aus vier vollvernetzten Schichten (`Linear`) mit LeakyReLU-Aktivierungen.
- Nimmt ein **eingeflattetes Bild (28x28) + One-Hot-Label (10 Klassen)** als Eingabe.
- Gibt `latent_dimension * 2` Werte aus: 
  - Erste Hälfte: **Mittelwert (μ)**
  - Zweite Hälfte: **Log-Varianz (log(σ²))** – für die Reparametrisierung.

#### Decoder
- Besteht aus vier `Linear`-Schichten mit LeakyReLU.
- Nimmt einen latent-Vektor + One-Hot-Label und rekonstruiert ein Bild (28x28).


### Vorwärtsdurchlauf (`forward`)
- Gibt direkt den rohen Output des Encoders zurück (z. B. für Inferenz).


#### `reparametrization(mu, log_var)`
- Wendet den Reparametrisierungstrick an:  
  $$ z = \mu + \epsilon \cdot \sigma $$
  Damit ist das Sampling differenzierbar für das Training per Backpropagation.


#### `training_step`
Verarbeitung eines Batches während des Trainings:
1. Bilddaten + One-Hot-Labels kombinieren.
2. Durch den Encoder → erhält `mu` und `log_var`.
3. Sampling mit Reparametrisierung.
4. Kombiniertes Sample + Label → Decoder → Rekonstruiertes Bild (`x_hat`).
5. Berechnung der Verluste:
   - **Rekonstruktionsverlust**: Mittels MSE zwischen `x_hat` und Originalbild `x`.
   - **Kullback-Leibler Divergenz (KLD)**: Regularisiert die latente Verteilung gegen eine Normalverteilung.
6. Gesamtverlust = Skaliertes MSE + KLD.
7. Logging der Metriken für TensorBoard etc.


#### `validation_step`
- Gleiche Schritte wie im Training – nur ohne Backpropagation.
- Loggt separate Metriken für die Validierung.


### `configure_optimizers`
- Optimierer: **Adam** mit Lernrate 1e-3.


### Anwendung
Dieses Modell kann:
- Bilder komprimieren & wiederherstellen,
- latente Repräsentationen lernen,
- Klassenbewusst neue Bilder generieren (z. B. eine "4" zeichnen lassen).

**Typischer Use Case**: Generatives Modell auf dem MNIST-Datensatz oder ähnlichem.

In [ ]:
def one_hot_embedding(labels, num_classes):
    y = torch.eye(num_classes, device=labels.device)
    return y[labels]


class LitAutoEncoder(pl.LightningModule):
    def __init__(self, latent_dimension=32):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28 + 10, 2048),
            nn.LeakyReLU(),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(),
            nn.Linear(1024, 512),
            nn.LeakyReLU(),
            nn.Linear(512, latent_dimension * 2),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dimension + 10, 2048),
            nn.LeakyReLU(),
            nn.Linear(2048, 2048),
            nn.LeakyReLU(),
            nn.Linear(2048, 2048),
            nn.LeakyReLU(),
            nn.Linear(2048, 28 * 28),
        )

    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        embedding = self.encoder(x)
        return embedding

    def reparametrization(self, mu, log_var):
        std = torch.exp(0.5 * log_var).sqrt()  # standard deviation
        eps = torch.randn_like(std)  # `randn_like` as we need the same size
        sample = mu + (eps * std)  # sampling as if coming from the input space
        return sample

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop. It is independent of forward
        x, labels = batch
        onehot = one_hot_embedding(labels, 10)
        x = x.view(x.size(0), -1)

        m = torch.cat([x, onehot], 1)
        z = self.encoder(m)
        mu = z[:, :32]
        log_var = z[:, 32:]

        sample = self.reparametrization(mu, log_var)

        q = torch.cat([sample, onehot], 1)
        x_hat = self.decoder(q)

        recon_loss = F.mse_loss(x_hat, x)
        KLD = -0.5 * torch.mean(1 + log_var - mu.pow(2) - log_var.exp())

        ramp_weight = min([1, self.global_step / 2500])
        loss = (55000 / 256) * recon_loss + KLD
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, on_step=False)
        self.log("kl", KLD, on_epoch=True, prog_bar=True, on_step=False)
        self.log("recon", recon_loss, on_epoch=True, prog_bar=True, on_step=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, labels = batch
        onehot = one_hot_embedding(labels, 10)
        x = x.view(x.size(0), -1)

        m = torch.cat([x, onehot], 1)
        z = self.encoder(m)
        mu = z[:, :32]
        log_var = z[:, 32:]
        sample = self.reparametrization(mu, log_var)

        q = torch.cat([sample, onehot], 1)
        x_hat = self.decoder(q)

        recon_loss = F.mse_loss(x_hat, x)
        KLD = -0.5 * torch.mean(1 + log_var - mu.pow(2) - log_var.exp())

        loss = (55000 / 256) * recon_loss + KLD
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, on_step=False)
        self.log("val_kl", KLD, on_epoch=True, on_step=False)
        self.log("val_recon", recon_loss, on_epoch=True, on_step=False)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


# **Training eines Variational Autoencoders mit PyTorch Lightning**

In diesem Codeabschnitt wird der **`LitAutoEncoder`** auf dem **MNIST-Datensatz** trainiert – mit automatischem Logging und Validierung.


In [ ]:
from pytorch_lightning.loggers import CSVLogger

logger = CSVLogger("logs", name="vae")

dataset = MNIST(
    os.getcwd() + "data/datasets", download=True, transform=transforms.ToTensor()
)
train, val = random_split(dataset, [55000, 5000])

autoencoder = LitAutoEncoder(latent_dimension=32)

trainer = pl.Trainer(accelerator="auto", devices=1, max_epochs=10, logger=logger)
trainer.fit(
    autoencoder,
    DataLoader(train, batch_size=128, num_workers=10, pin_memory=True),
    DataLoader(val, batch_size=128, num_workers=10, pin_memory=True),
)


## **Neueste `metrics.csv`-Datei finden (Logging-Auswertung)**

Dieses Skript durchsucht das Log-Verzeichnis eines PyTorch Lightning Experiments (z. B. VAE) und findet automatisch die neueste `metrics.csv`-Datei für die Analyse der Trainingsmetriken.


### Zweck
Das Ziel ist es, **automatisch die aktuellste Version des Log-Ordners** zu identifizieren (z. B. `version_0`, `version_1`, ...), in dem Lightning seine Metriken speichert.


### Was passiert genau?

1. **`find_latest_metrics_path(base_log_dir)`**
   - Liest alle Unterordner im Log-Verzeichnis, die mit `version_` beginnen.
   - Sortiert diese Ordner nach ihrer Nummer (z. B. `version_0`, `version_1`, ...).
   - Wählt den Ordner mit der höchsten Nummer (= letzter Trainingslauf).
   - Gibt den Pfad zur zugehörigen Datei `metrics.csv` zurück.

2. **`base_log_dir = os.path.join("logs", "vae")`**
   - Setzt das Basisverzeichnis, in dem die Logs gespeichert wurden (z. B. durch den `CSVLogger` beim Training).

3. **`metrics_path = find_latest_metrics_path(...)`**
   - Ruft die oben beschriebene Funktion auf, um den Pfad zur neuesten Log-Datei zu bekommen.

4. **`print(...)`**
   - Gibt den Pfad zur Konsole aus, z. B.:
     ```
     Neueste metrics.csv gefunden unter: logs/vae/version_3/metrics.csv
     ```


**Anwendung:**  
Nützlich für Auswertungen mit Pandas, Plotten von Trainings- und Validierungsverlauf oder automatisierte Reports nach dem Training.

In [ ]:
import os
import pandas as pd


def find_latest_metrics_path(base_log_dir):
    version_dirs = [d for d in os.listdir(base_log_dir) if d.startswith("version_")]
    if not version_dirs:
        raise FileNotFoundError("Keine version_x Ordner im Log-Verzeichnis gefunden.")

    # Nach Versionsnummer sortieren
    version_dirs.sort(key=lambda x: int(x.split("_")[1]))
    latest_version = version_dirs[-1]

    return os.path.join(base_log_dir, latest_version, "metrics.csv")


# Hauptverzeichnis für Logs
base_log_dir = os.path.join("logs", "vae")
metrics_path = find_latest_metrics_path(base_log_dir)

print("Neueste metrics.csv gefunden unter:", metrics_path)


## **Bedingte Bildgenerierung mit einem trainierten VAE**

Dieser Code generiert **neue MNIST-Bilder** für jede Ziffer (0 bis 9), indem er den Decoder des VAE mit einem zufälligen Latent-Vektor kombiniert und mit einem festen Label (als One-Hot-Vektor) "füttert".


### Schleife über alle Ziffern (0–9)

Für jede Ziffer wird ein bedingter Datensatz generiert und visualisiert:

#### 1. One-Hot-Vektor für das Ziel-Label

- Erstellt `512` One-Hot-Vektoren (für z. B. Ziffer 5 steht an Position 5 eine 1).
- Ziel: Alle generierten Bilder sollen zur **gleichen Ziffer** gehören.

```python
onehot = torch.zeros(((16**2) * 2, 10))
onehot[:, digit] = 1.0
```


#### 2. Zufällige latente Vektoren erzeugen

- `512` zufällige Vektoren aus einer Normalverteilung.
- Repräsentieren unterschiedliche Varianten der Ziel-Ziffer im Latent-Space.

```python
z = torch.randn(((16**2) * 2, 32)).float()
```


#### 3. Latent-Vektor + Label kombinieren und durch Decoder schicken

- Kombination aus latenten Informationen (`z`) und klassenspezifischem Label (`onehot`).
- Decoder erzeugt `512` Bilder (Größe: 28×28) aus diesen zusammengesetzten Eingaben.
- `detach()` entfernt sie vom Rechen-Graphen, da keine Gradienten berechnet werden müssen.

```python
q = torch.cat([z, onehot], 1)
y = autoencoder.decoder(q).detach()
```


#### 4. Umformung & Visualisierung

- Bilder werden ins richtige Format gebracht.
- `make_grid`: Erstellt ein großes Rasterbild aus allen generierten Ziffern (32 pro Zeile).

```python
y = y.reshape((512, 1, 28, 28))
y_grid = make_grid(y_images, nrow=32)
```


#### 5. Anzeigen des Bildrasters

- Zeigt das generierte Raster für jede Ziffer (`digit`).
- Es sollte pro Label eine Sammlung realistisch aussehender Ziffern angezeigt werden, z. B. 512 unterschiedliche Versionen der „7“.

```python
plt.imshow(y_grid[0].cpu(), cmap="binary", vmin=0, vmax=1)
plt.title("Samples conditioned to the label [{}]".format(digit))
plt.show()
```


### Ziel

Dieses Verfahren demonstriert die **kontrollierte Generierung**:

> Der VAE erzeugt neue Bilder, die **gezielt** zu einer bestimmten Ziffer gehören, gesteuert durch den One-Hot-Vektor.

Perfekt zur **Visualisierung des latenten Raums** und zur **Evaluierung der Generierungsfähigkeit** des Modells.

In [ ]:
for digit in range(10):
    onehot = torch.zeros(((16**2) * 2, 10))
    onehot[:, digit] = 1.0
    z = torch.randn(((16**2) * 2, 32)).float()

    q = torch.cat([z, onehot], 1)
    y = autoencoder.decoder(q).detach()
    y = y.reshape((16**2) * 2, 1, 28, 28)
    y_images = list(iter(y))

    y_grid = make_grid(y_images, nrow=16 * 2)

    plt.figure(figsize=(12, 8))
    plt.imshow(y_grid[0].cpu(), cmap="binary", vmin=0, vmax=1)
    plt.title("Samples conditioned to the label [{}]".format(digit))
    plt.show()


## Where's the intelligence?

- Was garantiert, dass bei Variational Autoencodern eine sinnvolle Repräsentation der Daten im latenten Raum generiert wird?

- **Imports:** PyTorch- und torchvision-Module für Modellierung, Datenverarbeitung und Bildspeicherung werden geladen.
- **Batchgrösse:** Es wird eine Batchgrösse von `100` definiert.
- **MNIST-Datensatz:**
  - Trainingsdaten werden heruntergeladen und als Tensor transformiert.
  - Testdaten werden lokal vorausgesetzt und ebenfalls in Tensoren umgewandelt.
- **DataLoader:**
  - `train_loader`: Lädt Trainingsdaten zufällig (für besseres Training).
  - `test_loader`: Lädt Testdaten in fester Reihenfolge (für Evaluation).

In [ ]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(
    root="./data/mnist_data/",
    train=True,
    transform=transforms.ToTensor(),
    download=True,
)
test_dataset = datasets.MNIST(
    root="./data/mnist_data/",
    train=False,
    transform=transforms.ToTensor(),
    download=False,
)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=bs, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=bs, shuffle=False
)


- **Modell:** Es wird ein Variational Autoencoder (VAE) als `nn.Module` definiert.
- **Encoder:**
  - Besteht aus zwei ReLU-aktivierten Linear-Schichten (`fc1`, `fc2`).
  - Gibt Mittelwert `mu` (`fc31`) und Log-Varianz `log_var` (`fc32`) des latenten Raums aus.
- **Sampling:** 
  - Aus `mu` und `log_var` wird mit der Reparametrisierungstrick ein latenter Vektor `z` gezogen.
- **Decoder:**
  - Rekonstruiert den Eingabevektor aus `z` durch drei Linear-Schichten (`fc4`–`fc6`) mit ReLU und Sigmoid am Ende.
- **Forward-Pass:**
  - Input `x` wird auf 784-Dim flatten, durch Encoder geschickt, `z` gesampled und durch Decoder rekonstruiert.
  - Gibt `rekonstruktion`, `mu` und `log_var` zurück.
- **Modellinstanzierung:**
  - VAE wird mit Eingabedimension 784 (MNIST-Bilder), zwei versteckten Schichten und latentem Raum mit 2 Dimensionen erstellt.
  - Falls CUDA verfügbar ist, wird das Modell auf die GPU verschoben.

In [ ]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()

        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)

    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h)  # mu, log_var

    def sampling(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)  # return z sample

    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h))

    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var


# build model
vae = VAE(x_dim=784, h_dim1=512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()


In [ ]:
# zeige das Modell
vae


- **Optimizer:** Es wird der Adam-Optimizer für die Parameter des VAE verwendet.
- **Loss-Funktion:** Kombiniert zwei Komponenten:
  - **Rekonstruktionsverlust (BCE):** Binary Cross Entropy zwischen Eingabe `x` und Rekonstruktion `recon_x`.
  - **KL-Divergenz (KLD):** Misst die Abweichung der latenten Verteilung von einer Standardnormalverteilung.
- **Gesamtkosten:** Summe aus `BCE` und `KLD` wird zurückgegeben.

In [ ]:
optimizer = optim.Adam(vae.parameters())


# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction="sum")
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD


- **train-Funktion:** Führt einen Trainingsdurchlauf über alle Batches durch.
- **Modus:** Modell wird in Trainingsmodus versetzt (`vae.train()`).
- **Schleife über DataLoader:**
  - Daten werden auf die GPU verschoben.
  - Gradienten werden auf Null gesetzt (`optimizer.zero_grad()`).
  - Vorwärtsdurchlauf: Rekonstruktion + latente Parameter (`mu`, `log_var`).
  - Verlustberechnung mit Rekonstruktionsfehler + KL-Divergenz.
  - Backward-Pass und Optimierungsschritt.
  - Zwischenergebnisse werden regelmäßig ausgegeben (alle 100 Batches).
- **Epoch-Ausgabe:** Am Ende wird der durchschnittliche Verlust über alle Trainingsdaten ausgegeben.

In [ ]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()

        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)

        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item() / len(data),
                )
            )
    print(
        "====> Epoch: {} Average loss: {:.4f}".format(
            epoch, train_loss / len(train_loader.dataset)
        )
    )


- **test-Funktion:** Bewertet das Modell auf dem Testdatensatz.
- **Modus:** Modell wird in Evaluierungsmodus versetzt (`vae.eval()`), Dropout & BatchNorm deaktiviert.
- **Ohne Gradienten:** `torch.no_grad()` spart Speicher und Rechenzeit.
- **Schleife über Testdaten:**
  - Daten werden auf die GPU verschoben.
  - Vorwärtsdurchlauf liefert Rekonstruktion und latente Parameter.
  - Verlust pro Batch wird summiert.
- **Ergebnis:** Durchschnittlicher Testverlust über alle Daten wird ausgegeben.

In [ ]:
def test():
    vae.eval()
    test_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)

            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()

    test_loss /= len(test_loader.dataset)
    print("====> Test set loss: {:.4f}".format(test_loss))


- **Training über 50 Epochen:** 
  - Für jede Epoche wird:
    - Die `train()`-Funktion aufgerufen (Training des VAE).
    - Danach die `test()`-Funktion ausgeführt (Evaluation des Modells).

In [ ]:
for epoch in range(1, 51):
    train(epoch)
    test()


- **Output-Verzeichnis:** Ordner `./data/samples` wird erstellt, falls er noch nicht existiert.
- **Generierung & Visualisierung:**
  - In zwei Schleifendurchläufen werden jeweils 128 zufällige Punkte `z` aus dem latenten Raum (2D) erzeugt.
  - Der Decoder wandelt diese in Bilder um.
  - Die generierten Bilder werden:
    - Als PNG-Datei gespeichert (`save_image`).
    - Als Grid visualisiert (`make_grid` + `matplotlib`), je 16 Bilder pro Zeile.

In [ ]:
# Sicherstellen, dass der Ordner existiert
output_dir = "./data/samples"
os.makedirs(output_dir, exist_ok=True)

# Mehrere Beispiele generieren und plotten
with torch.no_grad():
    for i in range(2):  # 10 verschiedene Beispiele
        z = torch.randn(128, 2).cuda()  # Mehr zufällige Punkte im latenten Raum
        sample = vae.decoder(z).cuda()  # Bilder generieren

        # Bild speichern
        save_image(sample.view(128, 1, 28, 28), f"{output_dir}/sample_{i}.png")

        # Bilder plotten
        grid = make_grid(
            sample.view(128, 1, 28, 28), nrow=16, normalize=True
        )  # Größeres Grid
        plt.figure(figsize=(12, 12))  # Größere Figur
        plt.imshow(grid.permute(1, 2, 0).cpu().numpy(), cmap="gray")
        plt.title(f"Generated Samples - Batch {i + 1}")
        plt.axis("off")
        plt.show()


- **Modell in Eval-Modus:** `vae.eval()` deaktiviert Trainingseffekte (z.B. Dropout).
- **Latent Space Grid:**
  - Ein 2D-Gitter aus Punkten im latenten Raum wird definiert (30x30 Werte von -3 bis 3).
  - Für jedes Koordinatenpaar `(x, y)` wird ein latenter Vektor `z` erzeugt und vom Decoder in ein Bild umgewandelt.
- **Bild-Grid:**
  - Die Bilder werden zeilen- und spaltenweise zu einem großen Bildraster (`make_grid`) zusammengesetzt.
  - Dieses Raster zeigt eine visuelle Interpolation durch den latenten Raum.
- **Visualisierung & Speicherung:**
  - Das große Bildraster wird geplottet und als PNG gespeichert (`latent_space_walk_large.png`).

In [ ]:
vae.eval()
grid_size = 30
grid_x = np.linspace(-3, 3, grid_size)
grid_y = np.linspace(-3, 3, grid_size)

samples = []

with torch.no_grad():
    for y in grid_y:
        row = []
        for x in grid_x:
            z = torch.tensor([[x, y]], dtype=torch.float32).cuda()
            img = vae.decoder(z).view(1, 28, 28)
            row.append(img)
        samples.append(torch.cat(row, dim=0))

all_samples = torch.cat(samples, dim=0).unsqueeze(1)
grid = make_grid(all_samples, nrow=grid_size, normalize=True, padding=2)

# Plot anzeigen
plt.figure(figsize=(20, 20))  # Größer!
plt.imshow(grid.permute(1, 2, 0).cpu().numpy(), cmap="gray")
plt.axis("off")
plt.title("Großer Latent Space Walk")
plt.show()

# Optional speichern
save_image(grid, "data/latent_space_walk_large.png")
